### Import necessary Library

In [2]:
import io
import os
import sys
import math
#import tarfile
import multiprocessing

import scipy
import librosa
import soundfile as sf

import requests
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
from IPython.display import Audio, display

import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T

import parselmouth
from parselmouth.praat import call
from pathlib import Path

from metrics import AudioMetrics
from metrics import AudioMetrics2
#from Audio_metrics import AudioMetrics2
import noise_addition_utils
from pypesq import pesq
import shutil

print(torch.__version__)
print(torchaudio.__version__)

2.3.1+cpu
2.3.1+cpu


c:\Projects\Denoise-module-main\Denoise-module-main\FRCRN_denoise\noise_addition_utils.py:15: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [10]:
#-------------------------------------------------------------------------------
# Helper functions.
#-------------------------------------------------------------------------------

def denoise_audio(input_path, output_path, smoothing_factor):
    # Load the audio file
    snd = parselmouth.Sound(input_path)

    # Apply smoothing to reduce noise
    snd_denoised = snd.copy()
    #snd_denoised = call(snd_denoised, "Smooth", smoothing_factor)
    snd_denoised = call(snd_denoised, "Reduce noise", 0.0, 0.0, 0.025, 80.0, 10000.0, 40.0,-20, "Spectral-subtraction")
    # Save the denoised audio
    snd_denoised.save(output_path, "WAV")
def get_stats(waveform, sample_rate=None, src=None):
    max_ = waveform.max().numpy()
    min_ = waveform.min().numpy()
    mean_ = waveform.mean().numpy()
    std_ = waveform.std().numpy()
    return max_,min_,mean_,std_

def print_stats(waveform, sample_rate=None, src=None):
    if src:
        print("-" * 10)
        print("Source:", src)
        print("-" * 10)
    if sample_rate:
        print("Sample Rate:", sample_rate)
        print("Shape:", tuple(waveform.shape))
        print("Dtype:", waveform.dtype)
        print(f" - Max:     {waveform.max().item():6.3f}")
        print(f" - Min:     {waveform.min().item():6.3f}")
        print(f" - Mean:    {waveform.mean().item():6.3f}")
        print(f" - Std Dev: {waveform.std().item():6.3f}")
        print()
        print(waveform)
        print()
    # max_ = waveform.max().numpy()
    # min_ = waveform.min().numpy()
    # mean_ = waveform.mean().numpy()
    # std_ = waveform.std().numpy()
    # return max_,min_,mean_,std_

def plot_waveform(waveform, sample_rate, title="Waveform", xlim=None, ylim=None):
  waveform = waveform.numpy()

  num_channels, num_frames = waveform.shape
  time_axis = torch.arange(0, num_frames) / sample_rate

  figure, axes = plt.subplots(num_channels, 1)
  if num_channels == 1:
    axes = [axes]
  for c in range(num_channels):
    axes[c].plot(time_axis, waveform[c], linewidth=1)
    axes[c].grid(True)
    if num_channels > 1:
      axes[c].set_ylabel(f'Channel {c+1}')
    if xlim:
      axes[c].set_xlim(xlim)
    if ylim:
      axes[c].set_ylim(ylim)
  figure.suptitle(title)
  plt.show(block=False)

def plot_specgram(waveform, sample_rate, title="Spectrogram", xlim=None):
  waveform = waveform.numpy()
  num_channels, num_frames = waveform.shape
  time_axis = torch.arange(0, num_frames) / sample_rate
  figure, axes = plt.subplots(num_channels, 1)
  if num_channels == 1:
    axes = [axes]
  for c in range(num_channels):
    axes[c].specgram(waveform[c], Fs=sample_rate)
    if num_channels > 1:
      axes[c].set_ylabel(f'Channel {c+1}')
    if xlim:
      axes[c].set_xlim(xlim)
  figure.suptitle(title)
  plt.show(block=False)

def play_audio(waveform, sample_rate):
  waveform = waveform.numpy()
  num_channels, num_frames = waveform.shape
  if num_channels == 1:
    display(Audio(waveform[0], rate=sample_rate))
  elif num_channels == 2:
    display(Audio((waveform[0], waveform[1]), rate=sample_rate))
  else:
    raise ValueError("Waveform with more than 2 channels are not supported.")

def plot_spectrogram(spec, title=None, ylabel='freq_bin', aspect='auto', xmax=None):
  fig, axs = plt.subplots(1, 1)
  axs.set_title(title or 'Spectrogram (db)')
  axs.set_ylabel(ylabel)
  axs.set_xlabel('frame')
  im = axs.imshow(librosa.power_to_db(spec), origin='lower', aspect=aspect)
  if xmax:
    axs.set_xlim((0, xmax))
  fig.colorbar(im, ax=axs)
  plt.show(block=False)

def get_spectrogram(
    n_fft = 400,
    win_len = None,
    hop_len = None,
    power = 2.0,
):
  waveform, _ = get_speech_sample()
  spectrogram = T.Spectrogram(
      n_fft=n_fft,
      win_length=win_len,
      hop_length=hop_len,
      center=True,
      pad_mode="reflect",
      power=power,
  )
  return spectrogram(waveform)
def resample_wav_files(input_path, output_path, target_sr):
    # Create the output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)

    # Get a list of all WAV files in the input directory and its subfolders
    for root, dirs, files in os.walk(input_path):
        for file_name in files:
            if file_name.endswith('.wav'):
                # Read the input WAV file
                input_file = os.path.join(root, file_name)
                audio, sr = librosa.load(input_file, sr=target_sr)

                # Write the resampled audio to the output WAV file
                output_dir = os.path.join(output_path, os.path.relpath(root, input_path))
                os.makedirs(output_dir, exist_ok=True)
                output_file = os.path.join(output_dir, file_name)
                sf.write(output_file, audio, target_sr)
def copy_wav_files_to_single_directory(input_path, output_path):
    # Create the output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)

    # Get a list of all WAV files in the input directory and its subfolders
    for root, dirs, files in os.walk(input_path):
        for file_name in files:
            if file_name.endswith('.wav'):
                # Copy the WAV file to the output directory with the original filename
                input_file = os.path.join(root, file_name)
                output_file = os.path.join(output_path, file_name)
                shutil.copy2(input_file, output_file)

### Spectral Subtraction method

#### Define the path to denoise 

In [11]:
bahna_dataset = sorted(list(Path(r"C:\Projects\Denoise-module-main\Denoise-module-main\Test").rglob('*.wav')))
bahna_dataset[0]

WindowsPath('C:/Projects/Denoise-module-main/Denoise-module-main/Test/0001.1.wav')

In [12]:
# dataframe to record statistics
stat_bahna_spectral_subtraction = pd.DataFrame(columns=['Name','Max', 'Min', 'Mean','Std','Noise_level_before_denoised','Max_after_denoised', 'Min_after_denoised', 'Mean_after_denoised','Std_after_denoised','Noise_level_after_denoised'])
stat_bahna_spectral_subtraction

,Name,Max,Min,Mean,Std,Noise_level_before_denoised,Max_after_denoised,Min_after_denoised,Mean_after_denoised,Std_after_denoised,Noise_level_after_denoised


In [13]:
#Load file to denoise
for i in bahna_dataset:
    print(i)
    waveform, sample_rate = torchaudio.load(i)
    max_,min_,mean_,std_ = get_stats(waveform, sample_rate = sample_rate)
    # Calculate noise level
    if std_!=0 :
        noise_level = 20*( np.log10(std_/max_))
    else:
        noise_level = 0
    normalized_path = os.path.normpath(i)
    input_path = os.path.splitext(normalized_path)[0] + os.path.splitext(normalized_path)[1]
    denoised_path = os.path.splitext(normalized_path)[0] + "_denoised_spectral_subtraction" + os.path.splitext(normalized_path)[1]
    snd = parselmouth.Sound(input_path)
    snd_denoised = snd.copy()
    snd_denoised = call(snd_denoised, "Reduce noise", 0.0, 0.0, 0.025, 80.0, 10000.0, 40.0,noise_level, "Spectral-subtraction")
    # Save the denoised audio
    snd_denoised.save(denoised_path, "WAV")
    print(denoised_path)
    waveform_denoised, sample_rate_denoised = torchaudio.load(denoised_path)
    max_after,min_after,mean_after,std_after=get_stats(waveform_denoised, sample_rate=sample_rate_denoised)
    if std_after!=0 :
        noise_level_after = 10*( np.log10(std_after/max_after))
    else:
        noise_level_after = 0
    df = pd.DataFrame({"Name":[i],"Max":[max_],"Min":[min_],"Mean":[mean_],"Std":[std_],"Noise_level_before_denoised":[noise_level],'Max_after_denoised':[max_after], 'Min_after_denoised':[min_after], 'Mean_after_denoised':[mean_after],'Std_after_denoised':[std_after],'Noise_level_after_denoised':[noise_level_after]})
    stat_bahna_spectral_subtraction=pd.concat([stat_bahna_spectral_subtraction,df], ignore_index=True)
    

C:\Projects\Denoise-module-main\Denoise-module-main\Test\0001.1.wav
C:\Projects\Denoise-module-main\Denoise-module-main\Test\0001.1_denoised_spectral_subtraction.wav
C:\Projects\Denoise-module-main\Denoise-module-main\Test\0001.2.wav
C:\Projects\Denoise-module-main\Denoise-module-main\Test\0001.2_denoised_spectral_subtraction.wav
C:\Projects\Denoise-module-main\Denoise-module-main\Test\0001.3.wav
C:\Projects\Denoise-module-main\Denoise-module-main\Test\0001.3_denoised_spectral_subtraction.wav
C:\Projects\Denoise-module-main\Denoise-module-main\Test\0001.4.wav
C:\Projects\Denoise-module-main\Denoise-module-main\Test\0001.4_denoised_spectral_subtraction.wav
C:\Projects\Denoise-module-main\Denoise-module-main\Test\0001.5.wav
C:\Projects\Denoise-module-main\Denoise-module-main\Test\0001.5_denoised_spectral_subtraction.wav


C:\Users\nghna\AppData\Local\Temp\ipykernel_15908\3705012350.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  stat_bahna_spectral_subtraction=pd.concat([stat_bahna_spectral_subtraction,df], ignore_index=True)


In [14]:
stat_bahna_spectral_subtraction.head()

,Name,Max,Min,Mean,Std,Noise_level_before_denoised,Max_after_denoised,Min_after_denoised,Mean_after_denoised,Std_after_denoised,Noise_level_after_denoised
0,C:\Projects\Denoise-module-main\Denoise-module...,0.6552124,-0.7651062,-3.0624975e-05,0.07685376,-18.614340,0.65423584,-0.76345825,-2.699632e-08,0.07657743,-9.316335
1,C:\Projects\Denoise-module-main\Denoise-module...,0.55578613,-0.4998474,-3.9136925e-05,0.0951897,-15.326356,0.5375061,-0.47183228,1.0024613e-06,0.090305954,-7.746670
2,C:\Projects\Denoise-module-main\Denoise-module...,0.47247314,-0.53967285,-2.9462188e-05,0.048951853,-19.692160,0.44641113,-0.47470093,1.3798474e-05,0.03827656,-10.668021
3,C:\Projects\Denoise-module-main\Denoise-module...,0.4461975,-0.56170654,-4.4831922e-05,0.059409264,-17.513460,0.44207764,-0.55718994,-3.0327021e-06,0.05851719,-8.782151
4,C:\Projects\Denoise-module-main\Denoise-module...,0.32443237,-0.37664795,-1.7288974e-05,0.046413366,-16.889622,0.32235718,-0.3694458,1.4427932e-06,0.044849005,-8.565845


In [15]:
sample_rate

48000

### FRCRN method

This model use a fixed input with Sampling frequency is 16khz so we have to resample the input before we put it into our model

In [24]:
# Example usage resample to 16khz
input_directory = r'C:\Projects\Denoise-module-main\Denoise-module-main\Test'
output_directory = r'C:\Projects\Denoise-module-main\Denoise-module-main\testTarget'
target_sampling_rate = 16000

resample_wav_files(input_directory, output_directory, target_sampling_rate)

#### Define path to denoised

In [5]:
bahna_dataset_FRCRN = sorted(list(Path(r"C:\Projects\Denoise-module-main\Denoise-module-main\Test").rglob('*.wav')))
bahna_dataset_FRCRN[0]

WindowsPath('C:/Projects/Denoise-module-main/Denoise-module-main/Test/0001.1.wav')

In [26]:
stat_bahna_FRCRN = pd.DataFrame(columns=['Name','Max', 'Min', 'Mean','Std','Noise_level_before_denoised','Max_after_denoised', 'Min_after_denoised', 'Mean_after_denoised','Std_after_denoised','Noise_level_after_denoised'])
stat_bahna_FRCRN

,Name,Max,Min,Mean,Std,Noise_level_before_denoised,Max_after_denoised,Min_after_denoised,Mean_after_denoised,Std_after_denoised,Noise_level_after_denoised


In [ ]:
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks


In [6]:
for i in bahna_dataset_FRCRN:
    print(i)
    waveform, sample_rate = torchaudio.load(i)
    max_,min_,mean_,std_ = get_stats(waveform, sample_rate = sample_rate)
    if std_!=0 :
        noise_level = 10*( np.log10(std_/max_))
    else:
        noise_level = 0
    # df = pd.DataFrame({"Name":[i],"Std":[std_],"Noise_level":[noise_level]})
    # stat_bahna=pd.concat([stat_bahna,df], ignore_index=True)
    normalized_path = os.path.normpath(i)
    input_path = os.path.splitext(normalized_path)[0] + os.path.splitext(normalized_path)[1]
    denoised_path = os.path.splitext(normalized_path)[0] + "_denoised_FRCRN" + os.path.splitext(normalized_path)[1]
    
    # denoised audio    
    ans = pipeline(
        Tasks.acoustic_noise_suppression,
        model='./demo_model')
    result = ans(
        str(i),
        output_path=denoised_path)
    print(denoised_path)
    waveform_denoised, sample_rate_denoised = torchaudio.load(denoised_path)
    max_after,min_after,mean_after,std_after=get_stats(waveform_denoised, sample_rate=sample_rate_denoised)
    if std_after!=0 :
        noise_level_after = 10*( np.log10(std_after/max_after))
    else:
        noise_level_after = 0
    df_FRCRN = pd.DataFrame({"Name":[i],"Max":[max_],"Min":[min_],"Mean":[mean_],"Std":[std_],"Noise_level_before_denoised":[noise_level],'Max_after_denoised':[max_after], 'Min_after_denoised':[min_after], 'Mean_after_denoised':[mean_after],'Std_after_denoised':[std_after],'Noise_level_after_denoised':[noise_level_after]})
    stat_bahna_FRCRN=pd.concat([stat_bahna_FRCRN,df_FRCRN], ignore_index=True)

C:\Projects\Denoise-module-main\Denoise-module-main\Test\0001.1.wav


NameError: name 'get_stats' is not defined

In [29]:
stat_bahna_FRCRN.head()

,Name,Max,Min,Mean,Std,Noise_level_before_denoised,Max_after_denoised,Min_after_denoised,Mean_after_denoised,Std_after_denoised,Noise_level_after_denoised
0,D:\Master\denoise_FRCRN_resample\ITTF-LTHDT-CH...,0.40127563,-0.31082153,-1.4884367e-05,0.02736577,-11.662351,0.45220947,-0.33789062,-3.996443e-06,0.024485594,-12.664291
1,D:\Master\denoise_FRCRN_resample\ITTF-LTHDT-CH...,0.31600952,-0.33432007,-1.4867977e-05,0.024471296,-11.110432,0.29574585,-0.2623291,-2.0226918e-05,0.016821647,-12.450502
2,D:\Master\denoise_FRCRN_resample\ITTF-LTHDT-CH...,0.17416382,-0.1734314,-1.4398085e-05,0.02409477,-8.590352,0.30462646,-0.24615479,-1.7838076e-05,0.016721833,-12.604837
3,D:\Master\denoise_FRCRN_resample\ITTF-LTHDT-CH...,0.4972229,-0.41744995,-1.5246733e-05,0.01634957,-14.830447,0.17391968,-0.18292236,-3.1963084e-06,0.014651101,-10.744785
4,D:\Master\denoise_FRCRN_resample\ITTF-LTHDT-CH...,0.57547,-0.4179077,-1.4917162e-05,0.01583354,-15.604447,0.20446777,-0.21505737,-3.3472586e-06,0.016336245,-10.974725


### Evaluation (PESQ, STOI)

PESQ Score only support for sampling rate Fs = 8khz (defined as narrow band) or 16khz (defined as wide band) only so we have to resample it

In [54]:
# Example usage resample to 16khz
# input_directory = r'D:/Master/Thesis/FRCRN/Summary/Test/'
# output_directory = 'D:/Master/Thesis/FRCRN/Summary/Test resample'
# target_sampling_rate = 16000

# resample_wav_files(input_directory, output_directory, target_sampling_rate)

In [ ]:
# Example usage Datasets/clean_testset_wav
# input_directory = 'D:\Master\Thesis\FRCRN\CTV bana to add noise\Bình Định'
# output_file = 'D:\Master\Thesis\FRCRN\CTV bana to add noise\Bình Định only'

# copy_wav_files_to_single_directory(input_directory, output_file)

#### Example evaluate PESQ (wide band - 16khz) and STOI for 1 clean - denoised

In [69]:
m = 'D:/Master/Thesis/FRCRN/Summary/Test resample/0001.1.wav'
n = 'D:/Master/Thesis/FRCRN/Summary/Test resample/0001.1_denoised_FRCRN.wav'
waveform, sample_rate = torchaudio.load(m)
waveform_denoised, sample_rate_denoised = torchaudio.load(n)

!pip install torchmetrics

In [70]:
from torchmetrics.audio import PerceptualEvaluationSpeechQuality
# pesq = PerceptualEvaluationSpeechQuality(8000, 'nb')
# pesq(preds, target)

wb_pesq = PerceptualEvaluationSpeechQuality(sample_rate, 'wb')
wb_pesq(waveform_denoised,waveform).item()

4.433448314666748

In [71]:
from torchmetrics.audio.stoi import ShortTimeObjectiveIntelligibility
stoi = ShortTimeObjectiveIntelligibility(sample_rate, False)
stoi(waveform_denoised,waveform).item()

0.9990944266319275

#### To evaluate multiple sample at one

In [72]:
metrics = pd.DataFrame(columns=['PESQ','STOI'])

In [ ]:
for i in bahna_dataset_FRCRN:
    print(i)
    waveform, sample_rate = torchaudio.load(i)
    #normalized_path = os.path.normpath(i)
    denoised_path = os.path.splitext(i)[0] + "_denoised_FRCRN" + os.path.splitext(i)[1]
    waveform_denoised, sample_rate_denoised = torchaudio.load(denoised_path)
    print(denoised_path)
    wb_pesq = PerceptualEvaluationSpeechQuality(sample_rate, 'wb')
    stoi = ShortTimeObjectiveIntelligibility(sample_rate, False)
    metric = pd.DataFrame({"PESQ":[wb_pesq(waveform, waveform_denoised)],"STOI":[stoi(waveform, waveform_denoised)]})
    metrics = pd.concat([metrics,metric], ignore_index=True)

In [74]:
metrics

,PESQ,STOI
0,tensor(4.1133),tensor(0.9992)
1,tensor(1.9175),tensor(0.9248)
2,tensor(2.6582),tensor(0.9736)
3,tensor(2.6802),tensor(0.9624)
4,tensor(4.3995),tensor(0.9952)
